In [0]:
CREATE TABLE IF NOT EXISTS IDENTIFIER(:catalog || '.' || :schema || '.metadata') (
  path STRING,
  agreement_currency STRING,
  agreement_name STRING,
  agreement_type STRING,
  document_type STRING,
  effective_date STRING,
  expiry_date STRING,
  termination_for_convenience STRING
)
USING DELTA;

MERGE INTO IDENTIFIER(:catalog || '.' || :schema || '.metadata') AS target
USING (
  SELECT
    path,
    metadata.*
  FROM (
    SELECT
      path,
      from_json(AI_QUERY(
        "databricks-claude-sonnet-4-5",
        CONCAT(
          'You are a contract expert with the task of extracting key information.
          
          Extract the following fields in json format from the vendor contract below:
          - AgreementCurrency: The currency of the contract, should be CAD, EUR, USD, or INR
          - AgreementName: The name of the agreement, often followed by (the Agreement)
          - AgreementType: The type of the aggreement according to this guide:
              CC- Construction Contract
              NDA - Non Disclosure Agreement
              CA - Confidentiality Agreement
              MWA - Miscellaneous Work Agreement
              MSA - Master Supply Agreement or MSSA - Master Supply and Service Agreement
              EP - Engineering and Procurement Contract
              EPC - Engineering Procurement and Construction Contract
              T&C - Terms and Conditions
              CSA - Consulting Services Agreement
              MOU - Memorandum of Understanding
              MOA - Memorandum of Agreement
              OTHER - Not covered by above
              NONAGREEMENT - Not an agreement
          - DocumentType: The type of the document, could be an agreement or other document according to this list:
              AGREEMENT - An Agreement
              CO or COR - Change Order
              Rate tables or Rate Schedules
              CCO - Contract Change Order - depending on the content, this may be considered an amendment
              Collective Bargaining Agreements
              Contract Executive Summary
              CRAF (recommendation for award)
              Proposals
              Technical Drawings
              Other
          - EffectiveDate: The start or effective date of the contract, often listed near the beginning
          - ExpiryDate: The end or expirin. Also look at the TERM section of each contract in the text. It should contain verbage around number of years from the effective date of the contract. First identify the effecive date and then look at the Term, adding the END DATE as another date.
          - TerminationForConvenience: Whether there is a clause for termination
          
          Return the output in json format. If you are not confident about a field, return ''.

          CONTRACT INFORMATION',
          'Vendor Name:', vendor_name, '\n',
          'File Name:', file_name, '\n',
          'Text:', truncated, '\n'
        ),
        responseFormat => 'STRUCT<result:STRUCT<
          agreement_currency:STRING, 
          agreement_name:STRING, 
          agreement_type:STRING, 
          document_type:STRING, 
          effective_date:STRING, 
          expiry_date:STRING, 
          termination_for_convenience:STRING
          >>'
      ),
      'STRUCT<
          agreement_currency:STRING, 
          agreement_name:STRING, 
          agreement_type:STRING, 
          document_type:STRING, 
          effective_date:STRING, 
          expiry_date:STRING, 
          termination_for_convenience:STRING
        >'
      )
      as metadata
    FROM IDENTIFIER(:catalog || '.' || :schema || '.flat')
  )
) AS source
ON target.path = source.path
WHEN NOT MATCHED THEN INSERT *;